# Week 1, Day 2 Exercise Solution

This notebook provides the solution to the exercise from Day 2. The goal is to adapt the website summarizer from Day 1 to use a local model running on Ollama instead of the paid OpenAI API.

### Step 1: Add Project Directory to Python Path

Since this notebook is in a subdirectory, we need to add the parent directory (`week1`) to Python's path to allow it to find and import the `scraper` module.

In [2]:
import sys
import os

# Add the parent directory (week1) to the system path
# This is necessary so that the notebook can find the 'scraper' module
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

### Step 2: Imports

We import the necessary libraries. `OpenAI` is still needed as it provides the client, `fetch_website_contents` comes from our scraper, and `display` and `Markdown` are for showing the output nicely in the notebook.

In [3]:
from openai import OpenAI
from scraper import fetch_website_contents
from IPython.display import display, Markdown

### Step 3: Initialize the Ollama Client

This is the key change. We create an instance of the `OpenAI` client, but we configure it to connect to our local Ollama server by providing the `base_url`. The `api_key` is required by the library, but Ollama doesn't use it, so we can pass a placeholder string.

In [4]:
ollama_client = OpenAI(
    base_url='http://localhost:11434/v1',
    api_key='ollama' # required, but unused
)

### Step 4: Define Prompts and Message Structure

We reuse the same prompts and message-building function from Day 1. No changes are needed here, as the Ollama endpoint is compatible with the OpenAI message format.

In [5]:
system_prompt = """
You are a snarky assistant that analyzes the contents of a website
and provides a short, concise summary, ignoring text that might be navigation related.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.

"""

def messages_for(website_content):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website_content}
    ]

### Step 5: Create the Summarization Function

We adapt the `summarize` function. Instead of calling the default OpenAI client, it now uses our `ollama_client`. We also specify a local model that we have pulled with Ollama, such as `llama3.2`.

In [6]:
def summarize(url):
    # 1. Get website content
    website_content = fetch_website_contents(url)
    
    # 2. Call the Ollama API
    response = ollama_client.chat.completions.create(
        model="llama3.2", # Or another model you have pulled, e.g., 'llama3.2:1b'
        messages=messages_for(website_content)
    )
    
    # 3. Return the content
    return response.choices[0].message.content

### Step 6: Display the Summary

Finally, we use a helper function to call our `summarize` function and display the result as formatted Markdown.

In [7]:
def display_summary(url):
    print(f"Fetching and summarizing {url}...\n")
    try:
        summary = summarize(url)
        display(Markdown(summary))
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please ensure your Ollama server is running and the model ('llama3.2') is pulled.")

### Step 7: Run the Example

Let's try it out on a simple website. Make sure your Ollama server is running in the background (`ollama serve`).

In [9]:
display_summary("http://edwarddonner.com")

Fetching and summarizing http://edwarddonner.com...



# Website Recap
The website appears to be a personal blog for Edward Donner, the co-founder and CTO of Nebula.io. The site offers resources on AI, including articles and upcoming events related to AI in production and deployment.

## Upcoming Events
* November 11, 2025: "The Unique Energy of an AI Live Event"
* September 15, 2025: "AI in Production: Gen AI and Agentic AI on AWS at scale"
* May 28, 2025: "Be an AI Engineer and Leader: The Curriculum"
* May 18, 2025: 2025 AI Executive Briefing